In [ ]:
# The following file is being used to test the Pipeline
''' Import statements for all the files involved'''
%matplotlib inline
from main import *
from utils import *
from wfield_utils import *
from registration import *
from hemocorrection import *
from denoising import *
from debug_visualize import *
from data_loading_functions import *
from data_preprocessor import *
from tqdm import tqdm

In [ ]:
# The folllowing code is from main(). 
# params are global variables used to modify the code
params = {
    'path_to_session_data':       r"C:\Users\evilm\Downloads\07-18-2024",
    'pipe_num':           0,              # select desired pipeline number
    'get_raw_brightness': False,
    'get_DeltaF':         False,
    'get_zScore':         False
          }
# This is inside def main()
# For purposes of testing to visualise the data we will do analysis here.
session_file_paths = get_file_paths(params['path_to_session_data'])
# create an obj for each recording within the session
recordings = {}
if session_file_paths != 0:
    n = 1
    recordings[f'recording_{n}'] = rawDataPreprocessor(params['path_to_session_data'])

print(f'{datetime.datetime.now().time()}: Done loading session data into object')


In [ ]:
#get_recording_paths(params['path_to_session_data'])
data = recordings['recording_1'].demo_pipeline()
data = np.array(data)
np.save(params['path_to_session_data']+"blue_frame_data",data)

In [ ]:
#plt.imshow(recordings['recording_1'].blue_frames[0],cmap='gray')
#plt.show()

In [ ]:
#plt.imshow(recordings['recording_1'].violet_frames[0],cmap='gray')
#plt.show()